## Importing the Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import yaml
import re
from tqdm import tqdm
import multiprocessing as mp
import time
import random
import datetime

## Importing the dataset

- Reading the 1 million playlists and keeping only the unique track URIs for the content-based recommendation system.
- The first for loop Read the 600 JSON files one at a time.
- The second for loop is for getting only the unique track.

In [6]:
def loop_slices(path, num_slices=20):
  cnt = 0
  cnt1 = 0
  mpd_playlists = []
  unique_tracks= pd.DataFrame()
  filenames = os.listdir(path)
  for fname in tqdm(sorted(filenames, key=len)):
    if fname.startswith("mpd.slice.") and fname.endswith(".json"):
      cnt += 1
      fullpath = os.sep.join((path, fname))
      f = open(fullpath)
      js = f.read()
      f.close()
      current_slice = json.loads(js)
      # Create a list of all playlists
      for playlist in current_slice['playlists']:
        cnt1 +=1
        mpd_playlists.append(playlist)
        if cnt1 == 600:
          cnt1=0
          temp=pd.DataFrame(mpd_playlists)
          temp=temp.explode('tracks')
          temp=pd.DataFrame(temp['tracks'].apply(pd.Series))
          unique_tracks=pd.concat([unique_tracks,temp],axis=0)
          unique_tracks.drop_duplicates(subset=['track_uri'],inplace=True)
          mpd_playlists = []
      if cnt == num_slices:
        break
  return unique_tracks
# Path where the json files are extracted
path = 'data/'

In [7]:
df = loop_slices(path, num_slices=600)
df.to_csv('data/1m.csv')

100%|█████████████████████████████████████████████████████████████████████████████▊| 599/600 [3:16:12<00:19, 19.65s/it]


In [18]:
df["track_uri"] = df["track_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])
df["artist_uri"] = df["artist_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])
df["album_uri"] = df["album_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])

In [19]:
df_new=df[['track_uri', 'artist_uri', 'album_uri']]

In [20]:
df_new.to_csv("1m_dataset.csv")

In [21]:
df=df_new

In [24]:
df

,track_uri,artist_uri,album_uri
0,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,6vV5UrXcfyQD1wu4Qo2I9K
1,6I9VzXrHxO9rA9A5euc8Ak,26dSoYclwsYLMAKD3tpOr4,0z7pVBGOD7HCIB7S8eLkLI
2,0WqIKmW4BTrj3eJFmnCKMv,6vWDO969PvNqNYHIOW5v0m,25hVFAxTlDvXbx2X2QkUkE
3,1AWQoqb9bSvzTjaLralEkT,31TPClRtHm23RisEBtV3X7,6QPkyl04rXwTGlGlcYaRoW
4,1lzr43nnXAijIGYnCT8M8H,5EvFsr3kj42KNv97ZEnqij,6NmFmPX56pcLBOFMhIiKvF
...,...,...,...
1760488,5qGlyp4XgwL0VV6vOkWV5S,1PpMfsOyfZ3nmcSoiIohpC,0DOrj4NjLZuUIlQqHKLsTr
1760489,5LSOkBJJsLu4SQwvyacIGp,0ty0xha1dbprYIUAQufkFn,6BTzoWgZBEIw4wddGcIsy6
1760490,5syAfgDKuppPBIHXLY6zBn,1KeIof0zqga5ojkmOKg88P,1wLw41yiDGcZowrw3L3Bkd
1760491,5YiSmmV9FX2pluBqbGVEta,05lXn8pr4Rm6gFD8zGi3ve,1lajhds6xa01ZEH5L7QAUX


In [22]:
t_uri=df["track_uri"].unique()
a_uri=df["artist_uri"].unique()